In [1]:
# -*- coding: utf-8 -*-


import os
import json
import re
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from math import log2
from google.colab import drive


print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU device count:", torch.cuda.device_count())


CUDA available: True
CUDA version: 12.4
GPU device count: 1


In [2]:

# ✅ 挂载 Google Drive
drive.mount('/content/drive')

# ✅ 只改这两个参数
start_index = 751
end_index = 780
range_tag = f"{start_index}-{end_index}"

# ✅ 自动拼接路径
BASE_PATH = "/content/drive/MyDrive/Cluster-proj"
INPUT_PATH = f"{BASE_PATH}/dataset/openai-gsm8k/train.jsonl"
OUTPUT_PATH = f"{BASE_PATH}/output/llm_steps/whole_logits/deepseek-math-7b-gsm-{range_tag}.json"


Mounted at /content/drive


In [3]:

# ✅ 加载数据子集
def load_dataset(path, start, end):
    subset = []
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if start <= i < end:
                subset.append(json.loads(line))
            if i >= end:
                break
    return subset

# ✅ 加载模型和 tokenizer
def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cuda",
        trust_remote_code=True,
        torch_dtype=torch.float16
    ).eval()
    return model, tokenizer

def top_k_logits_and_softmax(logits_tensor, k=3):
    topk = torch.topk(logits_tensor, k)
    topk_indices = topk.indices.cpu().tolist()
    topk_logits = topk.values.cpu().tolist()
    topk_softmax = F.softmax(logits_tensor, dim=-1)[topk.indices].cpu().tolist()

    # ✅ 计算 entropy（以 2 为底）
    entropy = -sum(p * log2(p) for p in topk_softmax if p > 0)

    # ✅ 计算信息量（-log2(p)）列表
    info_list = [-log2(p) if p > 0 else 0.0 for p in topk_softmax]

    return {
        "indices": topk_indices,
        "logits": topk_logits,
        "softmax": topk_softmax,
        "entropy": entropy,
        "information_content": info_list
    }

# ✅ 生成回答，保存 token-level top-3 logits
def generate_with_topk_logits(model, tokenizer, question):
    prompt = (
        f"You are a strict math tutor.\n"
        f"Answer the following math question with clear, step-by-step reasoning.\n\n"
        f"Your whole reasoning steps must be enclosed in : <steps> ... </steps>.\n\n"
        f"The final answer must be enclosed in <final_result> ... </final_result> tags.\n\n"
        f"Only use the tags: <steps>, </steps>, and <final_result>, </final_result>. Do not create or use any other tags.\n\n"
        f"Do not skip any step tags. Each tag must have a matching opening and closing pair.\n\n"
        f"If you break this rule, your answer will be considered invalid.\n\n"
        f"The final answer should be concise — a number or a few words only, not a full sentence.\n\n"
        f"Keep the entire response under 400 tokens.\n\n"
        f"Here is one example for your reference.\n\n"
        f"Example Question: Sarah has 3 packs of pencils. Each pack contains 12 pencils. "
        f"She gives away 15 pencils to her classmates. How many pencils does she have left?\n"
        f"Example Answer:\n"
        f"<steps>Sarah has 3 packs of pencils. Each pack contains 12 pencils, so 3 * 12 = 36 pencils.\n"
        f"She gives away 15 pencils to her classmates.\n"
        f"Subtract the pencils given away from the total: 36 - 15 = 21.</steps>\n"
        f"<final_result>21</final_result>\n\n"
        f"Your style should be similar to the above example style.\n"
        f"Here is your Question: {question}\nYour Answer with reasoning steps and final result:\n\n"
    )

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Get input hidden states (optional)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)
        input_hidden_states = outputs.hidden_states[-1][0]  # shape: (input_len, hidden_dim)

    # Generate output
    output = model.generate(
        **inputs,
        max_new_tokens=1000,
        return_dict_in_generate=True,
        output_scores=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    input_len = inputs['input_ids'].shape[-1]
    generated_ids = output.sequences[0][input_len:]
    raw_tokens = tokenizer.convert_ids_to_tokens(generated_ids)
    clean_tokens = [tokenizer.convert_tokens_to_string([tok]).strip() for tok in raw_tokens]

    # Forward pass to get hidden states for generated tokens
    with torch.no_grad():
        full_inputs = torch.cat([inputs['input_ids'], generated_ids.unsqueeze(0)], dim=1)
        full_attention = torch.ones_like(full_inputs)
        full_outputs = model(
            input_ids=full_inputs,
            attention_mask=full_attention,
            output_hidden_states=True,
            return_dict=True
        )
        full_hidden_states = full_outputs.hidden_states[-1][0]  # shape: (total_seq_len, hidden_dim)
        gen_hidden_states = full_hidden_states[input_len:]  # shape: (gen_len, hidden_dim)

    # Extract token-level logits, softmax, and hidden vector
    token_level = []
    for i, token in enumerate(clean_tokens):
        logits_tensor = output.scores[i][0]
        topk_info = top_k_logits_and_softmax(logits_tensor, k=3)

        chosen_id = generated_ids[i].item()
        chosen_prob = float(F.softmax(logits_tensor, dim=-1)[chosen_id].cpu())
        info_content = -log2(chosen_prob) if chosen_prob > 0 else 0.0

        token_entry = {
            "token": token,
            "chosen_prob": chosen_prob,
            "information_content": info_content,
            "topk_info": topk_info,
            "hidden_vector": gen_hidden_states[i].cpu().tolist()  # ✅ 保存 hidden state 向量
        }
        token_level.append(token_entry)

    answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    return answer, token_level


# ✅ 提取 <final_result>
def extract_final_result(text):
    match = re.search(r"<final_result>(.*?)</final_result>", text, re.DOTALL)
    return match.group(1).strip() if match else None

def extract_true_final_result(ans_str):
    match = re.search(r'####\s*(\S+)', ans_str)
    return match.group(1).strip() if match else None

# ✅ 主执行函数
def run_generation_pipeline(start_index, end_index, input_path, output_path, model_name):
    dataset = load_dataset(input_path, start_index, end_index)
    model, tokenizer = load_model_and_tokenizer(model_name)
    results = {}

    for idx, item in enumerate(dataset):
        global_id = start_index + idx
        qid = f"q_{global_id}"
        question = item["question"]
        true_whole_answer = item.get("answer", "")
        true_final_result = extract_true_final_result(true_whole_answer)
        results[qid] = {
            "question": question,
            "true_whole_answer": true_whole_answer,
            "true_final_result": true_final_result
        }

        for i in range(3):
            try:
                full_answer, token_probs = generate_with_topk_logits(model, tokenizer, question)
                final_result = extract_final_result(full_answer)
                results[qid][f"sampling{i}"] = {
                    "whole_answer": full_answer,
                    "token_probs": token_probs,
                    "final_result": final_result
                }
            except Exception as e:
                print(f"[ERROR] Failed on {qid} sampling{i}: {e}")
                continue

    with open(output_path, "w") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved: {output_path}")


In [4]:
# ✅ 执行入口
run_generation_pipeline(
    start_index=start_index,
    end_index=end_index,
    input_path=INPUT_PATH,
    output_path=OUTPUT_PATH,
    # model_name="deepseek-ai/deepseek-llm-7b-chat"
    model_name="deepseek-ai/deepseek-math-7b-instruct"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

KeyboardInterrupt: 